In [2]:
# Inicialice app
import os, pyspark
from pyspark.sql import *
#os.chdir("/home/ubuntu")

spark = SparkSession.builder.appName("prueba").master("local[*]").getOrCreate()
sc = spark.sparkContext

In [3]:
# Creo la funcion para parsear las fechas
import datetime
import time

def formatoFecha(elemento):
    if elemento.endswith('Z') and elemento[10]=="T" and elemento[19]==".":
        output = elemento[0:10] + " " + elemento[11:19] + elemento[20:]
        output = output[:-1]
        
    elementoFormateado=time.strptime(output, "%Y-%m-%d %H:%M:%S%f")
    #return output
    return datetime.datetime(elementoFormateado[0],elementoFormateado[1],elementoFormateado[2],elementoFormateado[3],
                            elementoFormateado[4],elementoFormateado[5],elementoFormateado[6]) #.replace(tzinfo=pytz.utc)

In [4]:
# Probamos que funciona bien
import pytz
a="2018-06-25T10:42:56.366000Z"
b=formatoFecha(a).replace(tzinfo=pytz.utc)
b



datetime.datetime(2018, 6, 25, 10, 42, 56, tzinfo=<UTC>)

In [6]:
# Cargamos los datos desde el fichero

#ALEX: poner ruta relativa. Ni siquiera, cuando sea un .py no hará falta. Leers directamente gdax_data_007.gz
#import os
#os.chdir("C:/sssss/")
#spark.read.json("data_007.json")



#data_file = "C:/Users/Carlos/Desktop/Data Science/00 -TFM/9- Parsing & Modelling/gdax_data_0007.gz"
data_file = "/home/ubuntu/data/gdax/historic/gdax_data_0007.gz"
data = spark.read.json(data_file)

In [11]:
# ALEX: Convertir tipos

data.withColumn("price", data.price.cast("float"))

DataFrame[asks: array<array<string>>, bids: array<array<string>>, changes: array<array<string>>, channels: array<struct<name:string,product_ids:array<string>>>, client_oid: string, funds: string, last_trade_id: bigint, maker_order_id: string, order_id: string, order_type: string, price: float, product_id: string, reason: string, remaining_size: string, sequence: bigint, side: string, size: string, taker_order_id: string, time: string, trade_id: bigint, type: string]

In [10]:
f.astype

AttributeError: module 'pyspark.sql.functions' has no attribute 'astype'

In [9]:
import pyspark.sql.functions as f

In [6]:
# Creamos la funcion para parsear el RDD:
from pyspark.sql import Row
import datetime
def parseoOB(linea):
    numericPrice = linea[10]
    numericRemaningSize = linea[13]
    numericSize = linea[16]
    parsedTime = linea[18]

    if linea[10] != None:
        numericPrice = float(linea[10])
    if linea[13] != None:
        numericRemaningSize = float(linea[13])
    if linea[16] != None:
        numericSize = float(linea[16])
    if linea[18] != None:
        parsedTime = formatoFecha(linea[18])



    return(Row(
        asks=linea[0],
        bids = linea[1],  
        changes= linea[2],
        channels  = linea[3],
        client_oid    = linea[4],
        funds  = linea[5],
        last_trade_id  = linea[6],
        maker_order_id  = linea[7],
        order_id  = linea[8],
        order_type  = linea[9],
        price  = numericPrice,
        product_id  = linea[11],
        reason  = linea[12],
        remaining_size  = numericRemaningSize,
        sequence  = linea[14],
        side  = linea[15],
        size  = numericSize,
        taker_order_id  = linea[17],
        time  = parsedTime,
        #time  = linea[18],
        trade_id = linea[19],
        type = linea[20]
    ))
    

In [7]:
# dataParsed = data.rdd.map(lambda x: parseoOB(x))
# Dividimos los diferentes pares

# dataBTC_USD=dataParsed.filter(lambda x: x.product_id == "BTC-USD")
# dataBTC_EUR=dataParsed.filter(lambda x: x.product_id == "BTC-EUR")
# dataBTC_GPB=dataParsed.filter(lambda x: x.product_id == "BTC-GPB")

# dataETH_USD=dataParsed.filter(lambda x: x.product_id == "ETH-USD")
# dataETH_EUR=dataParsed.filter(lambda x: x.product_id == "ETH-EUR")

# dataLTC_USD=dataParsed.filter(lambda x: x.product_id == "LTC-USD")
# dataLTC_EUR=dataParsed.filter(lambda x: x.product_id == "LTC-EUR")

# dataLTC_BTC=dataParsed.filter(lambda x: x.product_id == "LTC-BTC")
# dataETH_BTC=dataParsed.filter(lambda x: x.product_id == "ETH-BTC")

In [8]:
dataParsedBTC_USD = data.rdd.filter(lambda x: x.product_id == "BTC-USD").map(lambda x: parseoOB(x))
dataParsedBTC_USD.persist()
dataParsedBTC_USD.take(5)

[Row(asks=[['8187.98', '0.62088654'], ['8187.99', '0.0014044'], ['8188.23', '0.00116018'], ['8189.8', '0.00213679'], ['8189.81', '0.00152621'], ['8189.82', '0.00189244'], ['8189.94', '0.00225869'], ['8189.97', '0.001'], ['8189.98', '0.00115979'], ['8189.99', '0.502'], ['8190', '1.87791613'], ['8190.07', '0.23683915'], ['8190.47', '0.18793545'], ['8190.93', '0.099'], ['8192.5', '0.21'], ['8192.97', '0.001'], ['8193.09', '0.001'], ['8193.23', '0.5'], ['8193.25', '1'], ['8193.26', '1'], ['8193.98', '0.911'], ['8194.99', '0.37162116'], ['8195', '4.66102875'], ['8195.39', '0.00103239'], ['8197.2', '0.004'], ['8198', '0.00115867'], ['8198.01', '0.00115863'], ['8198.02', '0.009'], ['8198.1', '0.00115873'], ['8198.14', '0.691853'], ['8199.3', '0.0011586'], ['8199.45', '0.00176831'], ['8199.59', '0.00115859'], ['8199.6', '0.00115827'], ['8199.61', '0.00225619'], ['8199.62', '0.0014023'], ['8199.63', '0.00115851'], ['8199.64', '0.00115855'], ['8199.65', '0.00213403'], ['8199.66', '0.00115852'], 

In [ ]:
dataParsedBTC_USD.filter(lambda x: x.type == "snapshot").take(35)

In [ ]:
dataParsedBTC_USD.filter(lambda x: x.type == "l2update").take(100)

# 1. Creamos los matchs cada segundo

In [9]:
dataParsedBTC_USD.filter(lambda x: x.type == "match").take(10)

[Row(asks=None, bids=None, changes=None, channels=None, client_oid=None, funds=None, last_trade_id=None, maker_order_id='a7379fe4-7764-4c1c-b272-be425208c3df', order_id=None, order_type=None, price=8187.98, product_id='BTC-USD', reason=None, remaining_size=None, sequence=6414948212, side='sell', size=0.03592222, taker_order_id='a5e146d4-faa6-4fac-80f2-d4eb8cd2222d', time=datetime.datetime(2018, 7, 26, 14, 35, 39, 3), trade_id=47397900, type='match'),
 Row(asks=None, bids=None, changes=None, channels=None, client_oid=None, funds=None, last_trade_id=None, maker_order_id='33091648-22de-4950-94d7-119c818ea9db', order_id=None, order_type=None, price=8187.97, product_id='BTC-USD', reason=None, remaining_size=None, sequence=6414948596, side='buy', size=0.004482, taker_order_id='480a9014-7b0c-40aa-8ce8-1c56d61573d2', time=datetime.datetime(2018, 7, 26, 14, 35, 39, 3), trade_id=47397901, type='match'),
 Row(asks=None, bids=None, changes=None, channels=None, client_oid=None, funds=None, last_tra

In [9]:
dataBTC_USD_match=dataParsedBTC_USD.filter(lambda x: x.type == "match" and x.size != None).map(lambda x: ((x.time), (x.size,x.price,1)))

In [ ]:
dataBTC_USD_match.take(5)

In [ ]:
dataParsedBTC_USD.filter(lambda x: x.type == "match" and x.size != None).select("time", "size", "prive")
dataParsedBTC_USD.filter(lambda x: x.type == "match" and x.size != None).groupBy("time").select()# ALEX

In [10]:
dataBTC_USD_matchReducedDEF=dataBTC_USD_match.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1], x[2]+y[2], y[1]))
dataBTC_USD_matchReducedDEF.take(10)

[(datetime.datetime(2018, 7, 26, 14, 35, 39, 3),
  (0.04040422, 16375.95, 2, 8187.97)),
 (datetime.datetime(2018, 7, 26, 14, 35, 40, 3),
  (1.0995028199999999, 106461.45, 13, 8189.99)),
 (datetime.datetime(2018, 7, 26, 14, 35, 44, 3), (0.02454037, 8190.0, 1)),
 (datetime.datetime(2018, 7, 26, 14, 35, 49, 3), (0.41908691, 8190.0, 1)),
 (datetime.datetime(2018, 7, 26, 14, 35, 51, 3),
  (0.16408165, 16380.0, 2, 8190.0)),
 (datetime.datetime(2018, 7, 26, 14, 35, 57, 3), (0.03592209, 8190.0, 1)),
 (datetime.datetime(2018, 7, 26, 14, 36, 6, 3),
  (1.2211960300000002, 32760.0, 4, 8190.0)),
 (datetime.datetime(2018, 7, 26, 14, 36, 9, 3),
  (0.023941169999999998, 40950.07, 5, 8190.07)),
 (datetime.datetime(2018, 7, 26, 14, 36, 13, 3), (0.006, 8190.06, 1)),
 (datetime.datetime(2018, 7, 26, 14, 36, 14, 3), (0.1239342, 8190.06, 1))]

In [11]:
### PARTE 2 - Añadiendo precio máximo y minimo

In [12]:
dataBTC_USD_matchPrice = dataParsedBTC_USD.filter(lambda x: x.type == "match" and x.size != None and x.time.day != None).map(lambda x: (x.time, x.price))
dataBTC_USD_matchPrice.take(5)

[(datetime.datetime(2018, 7, 26, 14, 35, 39, 3), 8187.98),
 (datetime.datetime(2018, 7, 26, 14, 35, 39, 3), 8187.97),
 (datetime.datetime(2018, 7, 26, 14, 35, 40, 3), 8187.98),
 (datetime.datetime(2018, 7, 26, 14, 35, 40, 3), 8187.98),
 (datetime.datetime(2018, 7, 26, 14, 35, 40, 3), 8187.99)]

In [13]:
import pyspark.sql.functions as F
DF_dataBTC_USD_matchPrice=dataBTC_USD_matchPrice.toDF(["time","price"])
DF_dataBTC_USD_matchMaxPrice=DF_dataBTC_USD_matchPrice.groupBy("time").agg(F.max("price").alias("max")).rdd.map(lambda x: ((x.time),x.max))
DF_dataBTC_USD_matchMinPrice=DF_dataBTC_USD_matchPrice.groupBy("time").agg(F.min("price").alias("min")).rdd.map(lambda x: ((x.time),x.min))
DF_dataBTC_USD_matchLastPrice=DF_dataBTC_USD_matchPrice.groupBy("time").agg(F.last("price").alias("last")).rdd.map(lambda x: ((x.time),x.last))


In [14]:
DF_dataBTC_USD_matchPrice.take(3)

[Row(time=datetime.datetime(2018, 7, 26, 14, 35, 39, 3), price=8187.98),
 Row(time=datetime.datetime(2018, 7, 26, 14, 35, 39, 3), price=8187.97),
 Row(time=datetime.datetime(2018, 7, 26, 14, 35, 40, 3), price=8187.98)]

In [15]:
dataBTC_USD_matchReduced=dataBTC_USD_match.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1], x[2]+y[2]))
dataBTC_USD_matchReduced.take(5)

[(datetime.datetime(2018, 7, 26, 14, 35, 39, 3), (0.04040422, 16375.95, 2)),
 (datetime.datetime(2018, 7, 26, 14, 35, 40, 3),
  (1.0995028199999999, 106461.45, 13)),
 (datetime.datetime(2018, 7, 26, 14, 35, 44, 3), (0.02454037, 8190.0, 1)),
 (datetime.datetime(2018, 7, 26, 14, 35, 49, 3), (0.41908691, 8190.0, 1)),
 (datetime.datetime(2018, 7, 26, 14, 35, 51, 3), (0.16408165, 16380.0, 2))]

In [16]:
dataBTC_USD_matchReducedMax=dataBTC_USD_matchReduced.leftOuterJoin(DF_dataBTC_USD_matchMaxPrice)
dataBTC_USD_matchReducedMaxMin=dataBTC_USD_matchReducedMax.leftOuterJoin(DF_dataBTC_USD_matchMinPrice)
dataBTC_USD_matchReducedDEF=dataBTC_USD_matchReducedMaxMin.leftOuterJoin(DF_dataBTC_USD_matchLastPrice)

In [17]:
dataBTC_USD_matchReducedDEF.take(10)

[(datetime.datetime(2018, 7, 26, 16, 0, 29, 3),
  ((((0.00582503, 8201.01, 1), 8201.01), 8201.01), 8201.01)),
 (datetime.datetime(2018, 7, 26, 17, 25, 19, 3),
  ((((0.0409137, 16429.97, 2), 8214.99), 8214.98), 8214.99)),
 (datetime.datetime(2018, 7, 26, 14, 48, 30, 3),
  ((((0.02335243, 8195.0, 1), 8195.0), 8195.0), 8195.0)),
 (datetime.datetime(2018, 7, 26, 15, 46, 47, 3),
  ((((0.01573642, 24626.100000000002, 3), 8208.7), 8208.7), 8208.7)),
 (datetime.datetime(2018, 7, 26, 15, 34, 9, 3),
  ((((0.17114325, 8194.99, 1), 8194.99), 8194.99), 8194.99)),
 (datetime.datetime(2018, 7, 26, 15, 22, 7, 3),
  ((((0.05971058, 8204.98, 1), 8204.98), 8204.98), 8204.98)),
 (datetime.datetime(2018, 7, 26, 15, 55, 55, 3),
  ((((0.1269072, 16400.010000000002, 2), 8200.01), 8200.0), 8200.01)),
 (datetime.datetime(2018, 7, 26, 17, 28, 56, 3),
  ((((1.76513052, 16435.06, 2), 8217.53), 8217.53), 8217.53)),
 (datetime.datetime(2018, 7, 26, 16, 41, 48, 3),
  ((((0.03038493, 8186.83, 1), 8186.83), 8186.83), 8

In [18]:
dataBTC_USD_matchReducedDEFformated=dataBTC_USD_matchReducedDEF.mapValues(lambda x: x[0] + (x[1], )).map(lambda x: (x[0],x[1][0][0][0],x[1][0][0][1]/x[1][0][0][2],x[1][0][0][2], x[1][0][1],  x[1][1],  x[1][2]))#.map(lambda x: (x[0],x[1][0][0],x[1][0][1]/x[1][0][2],x[1][0][2],x[1][0][3],x[1][1],x[1][2],x[1][3]))
dataBTC_USD_matchReducedDEFformated.take(5)

[(datetime.datetime(2018, 7, 26, 16, 0, 29, 3),
  0.00582503,
  8201.01,
  1,
  8201.01,
  8201.01,
  8201.01),
 (datetime.datetime(2018, 7, 26, 17, 25, 19, 3),
  0.0409137,
  8214.985,
  2,
  8214.99,
  8214.98,
  8214.99),
 (datetime.datetime(2018, 7, 26, 14, 48, 30, 3),
  0.02335243,
  8195.0,
  1,
  8195.0,
  8195.0,
  8195.0),
 (datetime.datetime(2018, 7, 26, 15, 46, 47, 3),
  0.01573642,
  8208.7,
  3,
  8208.7,
  8208.7,
  8208.7),
 (datetime.datetime(2018, 7, 26, 15, 34, 9, 3),
  0.17114325,
  8194.99,
  1,
  8194.99,
  8194.99,
  8194.99)]

In [19]:
DFdataBTC_USD_matchReducedDEF=dataBTC_USD_matchReducedDEFformated.toDF(["time","size", "price_mean", "num_orders","price_max","price_min","price_last"])

In [20]:
from pyspark.sql.functions import col
DFdataBTC_USD_matchReducedDEFSorted=DFdataBTC_USD_matchReducedDEF.sort(col("time").asc())
DFdataBTC_USD_matchReducedDEFSorted.take(5)

[Row(time=datetime.datetime(2018, 7, 26, 14, 35, 39, 3), size=0.04040422, price_mean=8187.975, num_orders=2, price_max=8187.98, price_min=8187.97, price_last=8187.97),
 Row(time=datetime.datetime(2018, 7, 26, 14, 35, 40, 3), size=1.0995028199999999, price_mean=8189.342307692307, num_orders=13, price_max=8189.99, price_min=8187.98, price_last=8189.99),
 Row(time=datetime.datetime(2018, 7, 26, 14, 35, 44, 3), size=0.02454037, price_mean=8190.0, num_orders=1, price_max=8190.0, price_min=8190.0, price_last=8190.0),
 Row(time=datetime.datetime(2018, 7, 26, 14, 35, 49, 3), size=0.41908691, price_mean=8190.0, num_orders=1, price_max=8190.0, price_min=8190.0, price_last=8190.0),
 Row(time=datetime.datetime(2018, 7, 26, 14, 35, 51, 3), size=0.16408165, price_mean=8190.0, num_orders=2, price_max=8190.0, price_min=8190.0, price_last=8190.0)]

In [655]:
a=DFdataBTC_USD_matchReducedDEFSorted.rdd.map(lambda x : (str(x[0]),x[1],x[2],x[3],x[4],x[5],x[6]))
b=a.toDF()
c=b.toPandas()

In [656]:
c.to_csv("dataBTC_USD_0007.csv", index=False)

# 2. Order book

In [37]:
dataParsedBTC_USD.filter(lambda x: x.type == "snapshot").take(1)

[Row(asks=[['8187.98', '0.62088654'], ['8187.99', '0.0014044'], ['8188.23', '0.00116018'], ['8189.8', '0.00213679'], ['8189.81', '0.00152621'], ['8189.82', '0.00189244'], ['8189.94', '0.00225869'], ['8189.97', '0.001'], ['8189.98', '0.00115979'], ['8189.99', '0.502'], ['8190', '1.87791613'], ['8190.07', '0.23683915'], ['8190.47', '0.18793545'], ['8190.93', '0.099'], ['8192.5', '0.21'], ['8192.97', '0.001'], ['8193.09', '0.001'], ['8193.23', '0.5'], ['8193.25', '1'], ['8193.26', '1'], ['8193.98', '0.911'], ['8194.99', '0.37162116'], ['8195', '4.66102875'], ['8195.39', '0.00103239'], ['8197.2', '0.004'], ['8198', '0.00115867'], ['8198.01', '0.00115863'], ['8198.02', '0.009'], ['8198.1', '0.00115873'], ['8198.14', '0.691853'], ['8199.3', '0.0011586'], ['8199.45', '0.00176831'], ['8199.59', '0.00115859'], ['8199.6', '0.00115827'], ['8199.61', '0.00225619'], ['8199.62', '0.0014023'], ['8199.63', '0.00115851'], ['8199.64', '0.00115855'], ['8199.65', '0.00213403'], ['8199.66', '0.00115852'], 

In [21]:
asks = dataParsedBTC_USD.filter(lambda x: x.type == "snapshot").map(lambda x: (x[0],x[1])).map(lambda x: x[0]).collect()
bids = dataParsedBTC_USD.filter(lambda x: x.type == "snapshot").map(lambda x: (x[0],x[1])).map(lambda x: x[1]).collect()

In [327]:
# Convertimos la lista de listas a diccionario
asksDict = {float(t[0]):float(t[1]) for t in asks[0]}
bidsDict = {float(t[0]):float(t[1]) for t in bids[0]}


In [132]:
# Sacamos los cambios en bids y asks. Quitamos el string que nos inica Buy Sell al tenerlo en objetos separados
bidsChanges = dataParsedBTC_USD.filter(lambda x: x.type == "l2update").map(lambda x: ((x[18],x[2][0][0]),[round(float(x[2][0][1]),2),float(x[2][0][2])])).reduceByKey(lambda x,y: x+y).filter(lambda x: x[0][1] == "buy").map(lambda x: (x[0][0],x[1])).collect()
asksChanges = dataParsedBTC_USD.filter(lambda x: x.type == "l2update").map(lambda x: ((x[18],x[2][0][0]),[round(float(x[2][0][1]),2),float(x[2][0][2])])).reduceByKey(lambda x,y: x+y).filter(lambda x: x[0][1] == "sell").map(lambda x: (x[0][0],x[1])).collect()

In [619]:
bidsChangesDict = {t[0]:t[1] for t in bidsChanges}
asksChangesDict = {t[0]:t[1] for t in asksChanges}

In [624]:
bidsChangesDict

{datetime.datetime(2018, 7, 26, 14, 35, 38, 3): [8170.15,
  0.9,
  8170.15,
  0.0,
  8170.16,
  0.9,
  8183.07,
  1.8269,
  8183.18,
  2.31636,
  8168.52,
  0.0,
  8180.8,
  0.0,
  8170.16,
  0.0,
  8168.51,
  0.9,
  8170.15,
  0.9,
  7939.44,
  0.0064,
  7939.44,
  0.0042,
  7939.44,
  0.0027,
  7939.44,
  0.0017,
  8180.79,
  0.9,
  8173.97,
  0.0,
  7939.44,
  0.0,
  8187.97,
  17.00666345,
  8180.79,
  0.0,
  8168.51,
  0.0,
  8173.97,
  0.1,
  8180.8,
  0.9,
  8168.52,
  0.9,
  8170.15,
  0.0,
  8168.52,
  0.0,
  8168.51,
  0.9,
  8170.16,
  0.9,
  8172.61,
  0.0,
  8180.8,
  0.0,
  8170.16,
  0.0,
  8172.6,
  0.9,
  8180.79,
  0.9,
  8170.15,
  0.9,
  8180.79,
  0.0,
  8168.51,
  0.0,
  8168.52,
  0.9,
  8180.8,
  0.9,
  8168.52,
  0.0,
  8180.8,
  0.0,
  8168.51,
  0.9,
  8180.79,
  0.9],
 datetime.datetime(2018, 7, 26, 14, 35, 39, 3): [8168.51,
  0.0,
  8180.79,
  0.0,
  8180.8,
  0.9,
  8180.8,
  0.0,
  8168.52,
  0.9,
  8168.52,
  0.0,
  8180.79,
  0.9,
  8168.5,
  0.9,
  817

In [653]:
def gapMeasurement(asksDict, bidsDict,bidsChangesDict,asksChangesDict ) : 
    bidChangeTimeStepDic=dict()
    gap=[]
    all_timesteps = list(bidsChangesDict.keys() | asksChangesDict.keys())
    all_timesteps.sort()
    newBidDict = bidsDict
    newAskDict = asksDict
    
    for date in all_timesteps:
        
        bidChangeTimeStepDic=dict()
        askChangeTimeStepDic=dict()
        if date in bidsChangesDict.keys():
            bidChange = bidsChangesDict[date]
            
        else: 
            bidChange = [[]]
        
        if  date in asksChangesDict.keys():
            askChange = asksChangesDict[date]
        else:
            askChange = [[]]
        
        
        if len(bidChange) > 1:
            
            for i in range(len(bidChange)):
                
                if i%2 == 0:
                    bidChangeTimeStepDic[bidChange[i]] = {bidChange[i+1]}
                    #print(bidChange[0][i+1])
                    #print(bidChangeTimeStepDic)   
                   
                   
        #else: pass
        #print(askChange[0])
        if len(askChange) > 1:
            
            for i in range(len(askChange)):
                
                if i%2 == 0:
                    askChangeTimeStepDic[askChange[i]] = {askChange[i+1]}
       
        #else: pass
        # newBidDict = bidsDict

        for i in list(bidChangeTimeStepDic.keys()):
            #print(i)
#             print(newBidDict[i])
             #print(bidChangeTimeStepDic[i])

            newBidDict[i] = list(bidChangeTimeStepDic[i])[0]
        #print(list(newBidDict)[0:20])
        

    
        # newAskDict = asksDict
        #print(askChangeTimeStepDic)
        for i in list(askChangeTimeStepDic.keys()):
            #print(i)
            #print(newAskDict[i])

            newAskDict[i] = list(askChangeTimeStepDic[i])[0]
        #return(newAskDict)

        # ELIMINAR LOS PRECIOS CON VOLUMEN NULO
        newBidDict = { k : v for k,v in newBidDict.items() if v>0}
        newAskDict = { k : v for k,v in newAskDict.items() if v>0}
        print(list(newAskDict)[0:10])
        #return(newBidDict)
        #print(list(newBidDict)[0:20])
        # Creando el Gap para determinado timestep
        #print(date)
        lastAskThisTimeStep = list(newAskDict.keys())[0]
        lastBidThisTimeStep = list(newBidDict.keys())[0]
        #print(lastBidThisTimeStep)
        thisTimeStepGap= lastAskThisTimeStep - lastBidThisTimeStep
        #print(thisTimeStepGap)
    #print(list(newBidDict)[0:20])
    #gap=gap.append(thisTimeStepGap)
    
    #return(newAskDict)
        
    #return(newBidDict)            
            
        
            
# bidChangeTimeStepDic[bidChange[0][i]] 
        
        
    

In [654]:
a=gapMeasurement(asksDict, bidsDict,bidsChangesDict,asksChangesDict)
a

[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.

[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.

[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.

[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.

[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.02, 8211.0, 8212.11, 8212.12, 8212.16, 8212.47, 8212.5]
[8209.01, 8209.99, 8210.0, 8210.

KeyboardInterrupt: 

In [582]:
b.

In [298]:
list(bidsChangesDict.keys() & asksChangesDict.keys())

[datetime.datetime(2018, 7, 26, 15, 23, 25, 3),
 datetime.datetime(2018, 7, 26, 17, 26, 28, 3),
 datetime.datetime(2018, 7, 26, 17, 23, 16, 3),
 datetime.datetime(2018, 7, 26, 15, 34, 46, 3),
 datetime.datetime(2018, 7, 26, 16, 35, 11, 3),
 datetime.datetime(2018, 7, 26, 16, 33, 28, 3),
 datetime.datetime(2018, 7, 26, 16, 37, 8, 3),
 datetime.datetime(2018, 7, 26, 16, 17, 9, 3),
 datetime.datetime(2018, 7, 26, 16, 14, 26, 3),
 datetime.datetime(2018, 7, 26, 15, 47, 28, 3),
 datetime.datetime(2018, 7, 26, 16, 42, 39, 3),
 datetime.datetime(2018, 7, 26, 16, 19, 13, 3),
 datetime.datetime(2018, 7, 26, 15, 36, 35, 3),
 datetime.datetime(2018, 7, 26, 17, 1, 32, 3),
 datetime.datetime(2018, 7, 26, 16, 34, 26, 3),
 datetime.datetime(2018, 7, 26, 15, 6, 46, 3),
 datetime.datetime(2018, 7, 26, 15, 44, 50, 3),
 datetime.datetime(2018, 7, 26, 14, 58, 45, 3),
 datetime.datetime(2018, 7, 26, 15, 8, 46, 3),
 datetime.datetime(2018, 7, 26, 16, 43, 28, 3),
 datetime.datetime(2018, 7, 26, 17, 29, 11, 3

In [224]:
bidsChangesDict[datetime.datetime(2018, 7, 26, 15, 23, 25, 3)]

[[8195.02,
  0.0,
  8195.03,
  0.9,
  8195.03,
  0.0,
  8195.01,
  0.9,
  8195.01,
  0.0,
  8195.02,
  0.9,
  8195.02,
  0.0,
  8195.03,
  0.9,
  8195.03,
  0.0,
  8195.01,
  0.9,
  8195.01,
  0.0,
  8195.02,
  0.9,
  8195.02,
  0.0,
  8195.01,
  0.9,
  8195.01,
  0.0,
  8195.02,
  0.9,
  8195.02,
  0.0,
  8195.01,
  0.9,
  8195.01,
  0.0,
  8195.02,
  0.9,
  8195.02,
  0.0,
  8195.03,
  0.9,
  8195.03,
  0.0,
  8195.01,
  0.9,
  8195.01,
  0.0,
  8195.02,
  0.9,
  8195.02,
  0.0,
  8195.01,
  0.9,
  8195.01,
  0.0,
  8195.02,
  0.9,
  8195.02,
  0.0,
  8195.03,
  0.9,
  8195.03,
  0.0,
  8195.04,
  0.9,
  8185.74,
  0.0,
  8139.07,
  0.0,
  8165.74,
  0.0,
  8199.07,
  0.0,
  8159.07,
  0.0,
  8132.41,
  0.0,
  8195.04,
  0.0,
  8192.41,
  0.0,
  8179.07,
  0.0,
  8201.12,
  0.0,
  8172.41,
  0.0,
  8195.05,
  0.9,
  8145.74,
  0.0,
  8195.05,
  0.0,
  8201.07,
  0.001,
  8195.01,
  0.9,
  8195.01,
  0.0,
  8195.02,
  0.9,
  8195.02,
  0.0,
  8195.03,
  0.9,
  8195.03,
  0.0,
  8195.0

In [112]:
bidsChangesDict[datetime.datetime(2018, 7, 26, 17, 21, 49, 3)]

[6630.01, 0.0, 6599.01, 0.0015]

In [67]:
all_timesteps = list(bidsChangesDict.keys() | asksChangesDict.keys())
len(all_timesteps)

10954

In [108]:
datetime.datetime(2018, 7, 26, 17, 21, 50, 3) in bidsChangesDict.keys()


True

In [95]:
for i in range(len(all_timesteps)):
    print(bidsChangesDict[all_timesteps[i]])

[8195.02, 0.0, 8195.03, 0.9, 8195.03, 0.0, 8195.01, 0.9, 8195.01, 0.0, 8195.02, 0.9, 8195.02, 0.0, 8195.03, 0.9, 8195.03, 0.0, 8195.01, 0.9, 8195.01, 0.0, 8195.02, 0.9, 8195.02, 0.0, 8195.01, 0.9, 8195.01, 0.0, 8195.02, 0.9, 8195.02, 0.0, 8195.01, 0.9, 8195.01, 0.0, 8195.02, 0.9, 8195.02, 0.0, 8195.03, 0.9, 8195.03, 0.0, 8195.01, 0.9, 8195.01, 0.0, 8195.02, 0.9, 8195.02, 0.0, 8195.01, 0.9, 8195.01, 0.0, 8195.02, 0.9, 8195.02, 0.0, 8195.03, 0.9, 8195.03, 0.0, 8195.04, 0.9, 8185.74, 0.0, 8139.07, 0.0, 8165.74, 0.0, 8199.07, 0.0, 8159.07, 0.0, 8132.41, 0.0, 8195.04, 0.0, 8192.41, 0.0, 8179.07, 0.0, 8201.12, 0.0, 8172.41, 0.0, 8195.05, 0.9, 8145.74, 0.0, 8195.05, 0.0, 8201.07, 0.001, 8195.01, 0.9, 8195.01, 0.0, 8195.02, 0.9, 8195.02, 0.0, 8195.03, 0.9, 8195.03, 0.0, 8195.04, 0.9]
[8214.98, 42.88182273, 8214.98, 43.75553768, 6697.79, 0.0, 6663.24, 0.0015]
[8188.34, 7.177, 8133.91, 0.0, 8133.13, 15.24, 6599.01, 0.0, 6727.01, 0.0015]
[8194.99, 15.73711042, 8185.0, 2.23503659, 8180.0, 9.363311

KeyError: datetime.datetime(2018, 7, 26, 17, 21, 50, 3)

In [88]:
range(len(all_timesteps))

range(0, 10954)

In [22]:
a=DFdataBTC_USD_matchReducedDEFSorted.rdd.map(lambda x : (str(x[0]),x[1],x[2],x[3],x[4],x[5],x[6]))
b=a.toDF()
c=b.toPandas()


In [23]:
c.to_csv("dataBTC_USD_0007_gap.csv", index=False)